<center>
<img src="../../img/ods_stickers.jpg" />
    
## [mlcourse.ai](mlcourse.ai) – Open Machine Learning Course 

Author: [Yury Kashnitskiy](https://yorko.github.io). 
Translated and edited by [Maxim Keremet](https://www.linkedin.com/in/maximkeremet/), [Artem Trunov](https://www.linkedin.com/in/datamove/), and [Aditya Soni](https://www.linkedin.com/in/aditya-soni-0505a9124/). This material is subject to the terms and conditions of the [Creative Commons CC BY-NC-SA 4.0](https://creativecommons.org/licenses/by-nc-sa/4.0/) license. Free use is permitted for any non-commercial purpose.

# <center>Assignment #2. Fall 2018 <br> Exploratory Data Analysis (EDA) of US flights <br> (using Pandas, Matplotlib & Seaborn)

<img src='../../img/plane_sunset.png' width=50%>

Prior to working on the assignment, you'd better check out the corresponding course material:
 - [Visualization: from Simple Distributions to Dimensionality Reduction](https://mlcourse.ai/notebooks/blob/master/jupyter_english/topic02_visual_data_analysis/topic2_visual_data_analysis.ipynb?flush_cache=true)
 - [Overview of Seaborn, Matplotlib and Plotly libraries](https://mlcourse.ai/notebooks/blob/master/jupyter_english/topic02_visual_data_analysis/topic2_additional_seaborn_matplotlib_plotly.ipynb?flush_cache=true)
 - first lectures in [this](https://www.youtube.com/watch?v=QKTuw4PNOsU&list=PLVlY_7IJCMJeRfZ68eVfEcu-UcN9BbwiX) YouTube playlist 

### Your task is to:
 - write code and perform computations in the cells below
 - choose answers in the [webform](https://docs.google.com/forms/d/1qSTjLAGqsmpFRhacv0vM-CMQSTT_mtOalNXdRTcdtM0/edit)
 - submit answers with **the very same email and name** as in assignment 1. This is a part of the assignment, if you don't manage to do so, you won't get credits. If in doubt, you can re-submit A1 form till the deadline for A1, no problem
 
### <center> Deadline for A2: 2018 October 21, 20:59 CET



In [ ]:
import numpy as np
import pandas as pd
# pip install seaborn 
import seaborn as sns
import matplotlib.pyplot as plt

* Download the data [archive](http://stat-computing.org/dataexpo/2009/2008.csv.bz2) (Archived ~ 114 Mb, unzipped - ~ 690 Mb). No need to unzip - pandas can unbzip on the fly.
* Place it in the "../../data" folder, or change the path below according to your location.
* The dataset has information about carriers and flights between US airports during the year 2008. 
* Column description is available [here](http://www.transtats.bts.gov/Fields.asp?Table_ID=236). Visit this site to find ex. meaning of flight cancellation codes.

**Reading data into memory and creating a Pandas _DataFrame_ object**

(This may take a while, be patient)

We are not going to read in the whole dataset. In order to reduce memory footprint, we instead load only needed columns and cast them suitable data types.

In [ ]:
dtype = {'DayOfWeek': np.uint8, 'DayofMonth': np.uint8, 'Month': np.uint8 , 'Cancelled': np.uint8, 
         'Year': np.uint16, 'FlightNum': np.uint16 , 'Distance': np.uint16, 
         'UniqueCarrier': str, 'CancellationCode': str, 'Origin': str, 'Dest': str,
         'ArrDelay': np.float16, 'DepDelay': np.float16, 'CarrierDelay': np.float16,
         'WeatherDelay': np.float16, 'NASDelay': np.float16, 'SecurityDelay': np.float16,
         'LateAircraftDelay': np.float16, 'DepTime': np.float16}

In [ ]:
%%time
# change the path if needed
path = '../input/2008.csv.bz2'
flights_df = pd.read_csv(path, usecols=dtype.keys(), dtype=dtype)

**Check the number of rows and columns and print column names.**

In [ ]:
print(flights_df.shape)
print(flights_df.columns)

**Print first 5 rows of the dataset.**

In [ ]:
flights_df.head()

**Transpose the frame to see all features at once.**

In [ ]:
flights_df.head().T

**Examine data types of all features and total dataframe size in memory.**

In [ ]:
flights_df.info()

**Get basic statistics of each feature.**

In [ ]:
flights_df.describe().T

**Count unique Carriers and plot their relative share of flights:**

In [ ]:
flights_df['UniqueCarrier'].nunique()

In [ ]:
flights_df.groupby('UniqueCarrier').size().plot(kind='bar');

**We can also _group by_ category/categories in order to calculate different aggregated statistics.**

**For example, finding top-3 flight codes, that have the largest total distance travelled in year 2008.**

In [ ]:
flights_df.groupby(['UniqueCarrier','FlightNum'])['Distance'].sum().sort_values(ascending=False).iloc[:3]

**Another way:**

In [ ]:
flights_df.groupby(['UniqueCarrier','FlightNum'])\
  .agg({'Distance': [np.mean, np.sum, 'count'],
        'Cancelled': np.sum})\
  .sort_values(('Distance', 'sum'), ascending=False)\
  .iloc[0:3]

**Number of flights by days of week and months:**

In [ ]:
pd.crosstab(flights_df.Month, flights_df.DayOfWeek)

**It can also be handy to color such tables in order to easily notice outliers:**

In [ ]:
plt.imshow(pd.crosstab(flights_df.Month, flights_df.DayOfWeek),
           cmap='seismic', interpolation='none');

**Flight distance histogram:**

In [ ]:
flights_df.hist('Distance', bins=20);

**Making a histogram of flight frequency by date.**

In [ ]:
flights_df['Date'] = pd.to_datetime(flights_df.rename(columns={'DayofMonth': 'Day'})[['Year', 'Month', 'Day']])

In [ ]:
num_flights_by_date = flights_df.groupby('Date').size()

In [ ]:
num_flights_by_date.plot();

**Do you see a weekly pattern above? And below?**

In [ ]:
num_flights_by_date.rolling(window=7).mean().plot();

**1. Find top-10 carriers in terms of the number of completed flights (_UniqueCarrier_ column)?**

**Which of the listed below is _not_ in your top-10 list?**
- DL
- AA
- OO
- **EV **

In [ ]:
flights_df.groupby(['UniqueCarrier']).size().sort_values(ascending=False).iloc[:10]

**2. Plot distributions of flight cancellation reasons (_CancellationCode_).**

**What is the most frequent reason for flight cancellation? (Use this [link](https://www.transtats.bts.gov/Fields.asp?Table_ID=236) to translate codes into reasons)**
- carrier
- **weather condition**s 
- National Air System
- security reasons

In [ ]:
flights_df.groupby('CancellationCode').size().sort_values(ascending=False)

**3. Which route is the most frequent, in terms of the number of flights?**

(Take a look at _'Origin'_ and _'Dest'_ features. Consider _A->B_ and _B->A_ directions as _different_ routes) 

 - New-York – Washington
 - **San-Francisco – Los-Angeles** 
 - San-Jose – Dallas
 - New-York – San-Francisco

In [ ]:
flights_df.groupby(['Origin', 'Dest']).size().sort_values(ascending=False)

**4. Find top-5 delayed routes (count how many times they were delayed on departure). From all flights on these 5 routes, count all flights with weather conditions contributing to a delay.**

- 449 
- 539 
- 549 
- **668 **

In [ ]:
delay_df = flights_df[flights_df['DepDelay'] > 0].groupby(['Origin', 'Dest']).size().sort_values(ascending=False).head()

In [ ]:
flights = tuple(dict(delay_df).keys())
count = 0
for i in flights:
    count += flights_df[(flights_df['Origin'] == i[0]) & (flights_df['Dest'] == i[1]) & (flights_df['WeatherDelay'] > 0)]['WeatherDelay'].count()
print(count)

**5. Examine the hourly distribution of departure times. For that, create a new series from DepTime, removing missing values.**

**Choose all correct statements:**
 - Flights are normally distributed within time interval [0-23] (Search for: Normal distribution, bell curve).
 - Flights are uniformly distributed within time interval [0-23].
 - **In the period from 0 am to 4 am there are considerably less flights than from 7 pm to 8 pm.**

In [ ]:
dep = flights_df[flights_df['DepTime'].notna()][['DepTime', 'Year']]

dep['DepTime'] = dep.DepTime.apply(lambda x: int(x // 100) % 24)


In [ ]:
dep.groupby('DepTime').size().plot(kind='bar')

**6. Show how the number of flights changes through time (on the daily/weekly/monthly basis) and interpret the findings.**

**Choose all correct statements:**
- **The number of flights during weekends is less than during weekdays (working days).**
- The lowest number of flights is on Sunday.
- **There are less flights during winter than during summer.**  

In [ ]:
flights_df.groupby('DayOfWeek').size().plot(kind='bar') #weekly flights graph

In [ ]:
flights_df.groupby('Month').size().plot(kind='bar') #monthly graph

**7. Examine the distribution of cancellation reasons with time. Make a bar plot of cancellation reasons aggregated by months.**

**Choose all correct statements:**
- **December has the highest rate of cancellations due to weather. **
- The highest rate of cancellations in September is due to Security reasons.
- **April's top cancellation reason is carriers.**
- Flights cancellations due to National Air System are more frequent than those due to carriers.

In [ ]:
flights_df.groupby('Month')['CancellationCode'].value_counts().plot(kind='bar', figsize=(20,8));


In [ ]:
flights_df.groupby(['Month', 'CancellationCode']).size().unstack().plot(kind='bar')

**8. Which month has the greatest number of cancellations due to Carrier?** 
- May
- January
- September
- **April** 

In [ ]:
carrier_cancel = flights_df[flights_df['CancellationCode'] == 'A'].groupby(['Month'])['UniqueCarrier']\
                                        .count().plot(kind='bar')

**9. Identify the carrier with the greatest number of cancellations due to carrier in the corresponding month from the previous question.**

- 9E
- EV
- HA
- **AA**

In [ ]:
flights_df[(flights_df['CancellationCode'] == 'A') & (flights_df['Month'] == 4)].groupby('UniqueCarrier')['CancellationCode'].count().sort_values()

**10. Examine median arrival and departure delays (in time) by carrier. Which carrier has the lowest median delay time for both arrivals and departures? Leave only non-negative values of delay times ('ArrDelay', 'DepDelay').
[Boxplots](https://seaborn.pydata.org/generated/seaborn.boxplot.html) can be helpful in this exercise, as well as it might be a good idea to remove outliers in order to build nice graphs. You can exclude delay time values higher than a corresponding .95 percentile.**

- EV
- OO
- AA
- AQ 

In [ ]:
flights_df.groupby('UniqueCarrier')['ArrDelay'].median().sort_values()
flights_df.groupby('UniqueCarrier')['DepDelay'].median().sort_values()
sns.boxplot(x='UniqueCarrier', y='ArrDelay', data=flights_df)

In [ ]:
sub_flights_df = flights_df[(flights_df['ArrDelay'] > 0) & (flights_df['ArrDelay'] < 60) &
                            (flights_df['DepDelay'] > 0) & (flights_df['DepDelay'] < 60)]
sns.boxplot(x="UniqueCarrier", y="ArrDelay", data=sub_flights_df)

In [ ]:
sns.boxplot(x="UniqueCarrier", y="DepDelay", data=sub_flights_df)